In [1]:
# %%
import sklearn.datasets
import numpy as np
import pandas as pd
from tqdm import tqdm
import json, pprint, base64

def lprint(iterator_object: iter):
    for x in iterator_object:
        print(x)
# %%
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 500)
pd.set_option("max_colwidth", 500)

/var/folders/x6/tdg2_lyn14n9vxhrw5znyrtw0000gn/T/ipykernel_38975/4271010074.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [172]:
# %%
# %%bigquery fc_spec
# select *
# from analytics.stg_mysql_views
# WHERE type = 'fullyConnected' AND project_id IS NULL

from google.cloud import bigquery

client = bigquery.Client(project='wandb-production')

query = """
    SELECT *
    FROM `analytics.stg_mysql_views`
    WHERE type = 'fullyConnected' 
"""

#     -- AND project_id IS NULL

fc_spec = client.query(query).to_dataframe()

print(f"Query returned {len(fc_spec)} rows.")
fc_spec.head()

Query returned 1 rows.


,view_id,cover_url,created_at,created_using,description,display_name,is_locked,name,parent_id,preview_url,project_id,showcased_at,spec,type,updated_at,updated_by,user_id,view_count
0,609783,None,2021-04-14 13:28:09+00:00,WANDB_UI,None,None,True,,<NA>,None,<NA>,NaT,"{""categories"":[{""color"":""#1A1C1F"",""description"":"""",""featuredReportIDs"":[""VmlldzoxNjg1ODQ1"",""VmlldzoxNjcwOTg1"",""VmlldzoxNjY3MDY0"",""VmlldzoxNjkwNjIz""],""id"":""ql7j6wvw7"",""imageURL"":"""",""linkForm"":"""",""name"":""Featured"",""updatedAt"":""2022-03-16T17:05:57.591Z""},{""color"":""#FF01E4"",""description"":"""",""featuredReportIDs"":[""VmlldzoxMjM2MDcz"",""VmlldzoxMTcxMjM5"",""VmlldzoxNDMzNzgx"",""VmlldzoxNTA2MzQw"",""VmlldzoxNDc2MTEw"",""VmlldzoxNTA5MzUx""],""id"":""ir6df9nxe"",""imageURL"":"""",""linkForm"":""posts"",""name"":""Blog Posts"",""u...",fullyConnected,2023-12-17 19:45:03+00:00,191954,24307,0


In [173]:
from pprint import pprint
pprint(fc_spec["spec"].values)

array(['{"categories":[{"color":"#1A1C1F","description":"","featuredReportIDs":["VmlldzoxNjg1ODQ1","VmlldzoxNjcwOTg1","VmlldzoxNjY3MDY0","VmlldzoxNjkwNjIz"],"id":"ql7j6wvw7","imageURL":"","linkForm":"","name":"Featured","updatedAt":"2022-03-16T17:05:57.591Z"},{"color":"#FF01E4","description":"","featuredReportIDs":["VmlldzoxMjM2MDcz","VmlldzoxMTcxMjM5","VmlldzoxNDMzNzgx","VmlldzoxNTA2MzQw","VmlldzoxNDc2MTEw","VmlldzoxNTA5MzUx"],"id":"ir6df9nxe","imageURL":"","linkForm":"posts","name":"Blog Posts","updatedAt":"2022-02-24T21:27:30.172Z"},{"color":"#0094FF","description":"","featuredReportIDs":["VmlldzoxNjAxNjk2"],"id":"swntrsf8t","imageURL":"","linkForm":"events","name":"Events","updatedAt":"2022-02-28T23:01:55.650Z"},{"color":"#29C6B3","description":"","featuredReportIDs":["Vmlldzo2MjgzODg=","VmlldzoxNTkyNDI1"],"id":"s4jjx15g2","imageURL":"","linkForm":"gradient-dissent","name":"Gradient Dissent","updatedAt":"2022-03-10T00:13:04.979Z"},{"color":"#8E00FF","description":"","featuredReport

In [182]:
len(fc_spec["spec"].values)

1

In [185]:
sss = json.loads(fc_spec["spec"].values[0])  #.values

for s in sss:
    print(s)

categories
eventMetadatas
headings
reportIDsWithTagIDs
reportIDsWithTagV2IDs
repos
tags
tagsV2
topLevelFeaturedReportIDs


In [193]:
def extract_report_ids(data):
    report_ids = set()

    if isinstance(data, dict):
        for key, value in data.items():
            report_ids.update(extract_report_ids(value))
    elif isinstance(data, list):
        for item in data:
            report_ids.update(extract_report_ids(item))
    elif isinstance(data, str) and data.startswith("Vmlldz"):
        report_ids.add(data)

    return list(report_ids)

sss = json.loads(fc_spec["spec"].values[0])
tmp_sss = {k:v for k,v in sss.items() if k == "reportIDsWithTagIDs" or k == "reportIDsWithTagIDs"}
report_ids = extract_report_ids(tmp_sss)
print(len(report_ids))
print(report_ids[:10])

930
['Vmlldzo2MDM4OTI=', 'Vmlldzo4Njg0MTY=', 'Vmlldzo0Mzg0MTI=', 'Vmlldzo4MTc0MTA=', 'VmlldzoxNTEwNzIx', 'Vmlldzo4NjIxODA=', 'Vmlldzo2MjY4MDA=', 'VmlldzoxMzUwMjIy', 'VmlldzozMTkxOTMw', 'Vmlldzo0ODI5ODM=']


In [ ]:
# for s in sss:
#     print(s, sss[s])

In [186]:
print(sss)

{'categories': [{'color': '#1A1C1F', 'description': '', 'featuredReportIDs': ['VmlldzoxNjg1ODQ1', 'VmlldzoxNjcwOTg1', 'VmlldzoxNjY3MDY0', 'VmlldzoxNjkwNjIz'], 'id': 'ql7j6wvw7', 'imageURL': '', 'linkForm': '', 'name': 'Featured', 'updatedAt': '2022-03-16T17:05:57.591Z'}, {'color': '#FF01E4', 'description': '', 'featuredReportIDs': ['VmlldzoxMjM2MDcz', 'VmlldzoxMTcxMjM5', 'VmlldzoxNDMzNzgx', 'VmlldzoxNTA2MzQw', 'VmlldzoxNDc2MTEw', 'VmlldzoxNTA5MzUx'], 'id': 'ir6df9nxe', 'imageURL': '', 'linkForm': 'posts', 'name': 'Blog Posts', 'updatedAt': '2022-02-24T21:27:30.172Z'}, {'color': '#0094FF', 'description': '', 'featuredReportIDs': ['VmlldzoxNjAxNjk2'], 'id': 'swntrsf8t', 'imageURL': '', 'linkForm': 'events', 'name': 'Events', 'updatedAt': '2022-02-28T23:01:55.650Z'}, {'color': '#29C6B3', 'description': '', 'featuredReportIDs': ['Vmlldzo2MjgzODg=', 'VmlldzoxNTkyNDI1'], 'id': 's4jjx15g2', 'imageURL': '', 'linkForm': 'gradient-dissent', 'name': 'Gradient Dissent', 'updatedAt': '2022-03-10T00

In [178]:
# load the json as a dict
fc_dict = fc_spec["spec"].apply(json.loads) 
# fc_spec["spec"].values[0]
fc_dict

0    {'categories': [{'color': '#1A1C1F', 'description': '', 'featuredReportIDs': ['VmlldzoxNjg1ODQ1', 'VmlldzoxNjcwOTg1', 'VmlldzoxNjY3MDY0', 'VmlldzoxNjkwNjIz'], 'id': 'ql7j6wvw7', 'imageURL': '', 'linkForm': '', 'name': 'Featured', 'updatedAt': '2022-03-16T17:05:57.591Z'}, {'color': '#FF01E4', 'description': '', 'featuredReportIDs': ['VmlldzoxMjM2MDcz', 'VmlldzoxMTcxMjM5', 'VmlldzoxNDMzNzgx', 'VmlldzoxNTA2MzQw', 'VmlldzoxNDc2MTEw', 'VmlldzoxNTA5MzUx'], 'id': 'ir6df9nxe', 'imageURL': '', 'linkF...
Name: spec, dtype: object

In [ ]:
fc_dict[0]

In [16]:
type(fc_spec["spec"])

pandas.core.series.Series

In [ ]:

# # %%
# %%bigquery fc_df
# SELECT * 
# FROM EXTERNAL_QUERY('wandb-production.us.wandb-prod-replica', 
#   '''
#   select 
#     spec
#   from views
#   where type = 'fullyConnected' AND project_id IS NULL
#   '''
# )

In [7]:
# %%
def generate_tag_hierarchy(fc_spec):
    tags_v2 = json.loads(fc_spec['spec'][0])['tagsV2']

    tags_v2_hierarchy_dict = {} 

    for tag in tags_v2:
        if tag['parent'] == 'c5t7x7ylo':
            tags_v2_hierarchy_dict[tag['id']] = tags_v2_hierarchy_dict.get(
                tag['id'], 
                {
                    'name': tag['name'],
                    'children': []
                }
            )
        else:
            parent_tag_dict = tags_v2_hierarchy_dict.get(
                tag["parent"], {"name": tag["name"], "children": []}
            )
            parent_tag_dict["children"].append((tag["name"], tag["id"]))
            tags_v2_hierarchy_dict[tag["parent"]] = parent_tag_dict
    return tags_v2_hierarchy_dict

def start_pipeline(df):
    return df.copy()

def tags_to_long_df(fc_spec):
    tag_hierarchy = generate_tag_hierarchy(fc_spec)
    df = pd.json_normalize(
        [
            dict({'tagID': k}, **v) 
            for k,v in tag_hierarchy.items()
        ], 
        'children', 
        ['tagID', 'name']
    ).rename(
        columns = {
            1: 'tagID', 
            0: 'tagName', 
            'tagID': 'parentTagID', 
            'name': 'parentTagName'
        }
    )
    return df

def reports_to_long_df(fc_spec):
    reports_metadata = json.loads(fc_spec['spec'][0])['reportIDsWithTagV2IDs']
    df = pd.json_normalize(
        reports_metadata, 
        'tagIDs', 
        ["id", "authors"]
    ).rename(
        columns = {0: 'tagID', 'id': 'reportID'}
    )
    return df

def pivot_to_labels_matrix(reports, tags):
    df = tags.merge(
        reports, 
        on='tagID'
    ).pivot_table(
        index= "reportID", 
        columns= "parentTagName", 
        values= "tagName",
        aggfunc= lambda x:list(x)
    )
    return df

In [ ]:
def reports_to_long_df(fc_spec):
    reports_metadata = json.loads(fc_spec['spec'][0])['reportIDsWithTagV2IDs']
    df = pd.json_normalize(
        reports_metadata, 
        'tagIDs', 
        ["id", "authors"]
    ).rename(
        columns = {0: 'tagID', 'id': 'reportID'}
    )
    return df

reports_to_long_df(fc_spec).head()

In [9]:
tag_long = tags_to_long_df(fc_spec)
labels_matrix = (reports_to_long_df(fc_spec)
  .pipe(start_pipeline)
  .pipe(pivot_to_labels_matrix, tags=tag_long)
)
print(len(labels_matrix))
# print()
labels_matrix.head()

1609


parentTagName,Articles,Course,Data,Domain,GPT2,Gradient Dissent,Has Colab,Level,Model,Normalization,OpenAI,Other,Task,Technique(s) or Industry (etc),Type,W&B Features
reportID,,,,,,,,,,,,,,,,
Vmlldzo0MDAxNzc3,[ML News],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vmlldzo0MDE4MTY=,NaN,NaN,NaN,[Computer Vision],NaN,NaN,"[Yes-Colab, Github]",[Intermediate],NaN,NaN,NaN,[TMP],[GenAI],NaN,"[Experiment, Research]",[Panels]
Vmlldzo0MDI0NTYz,[ML News],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vmlldzo0MDI2OTc0,"[Articles, From The Community]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[PyTorch Lightning],NaN,NaN,NaN,NaN,NaN
Vmlldzo0MDIyNDIz,"[Articles, Podcast]",NaN,NaN,NaN,NaN,[Gradient Dissent],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:

# %%
from google.cloud import bigquery

client = bigquery.Client(project='wandb-production')

query = """
SELECT 
   created_at, 
   description,
   display_name, 
   is_public, 
   name, 
   report_id, 
   report_path, 
   showcased_at, 
   spec, 
   stars_count, 
   user_id, 
   view_count
FROM analytics.dim_reports
WHERE showcased_at IS NOT NULL
"""

views_df = client.query(query).to_dataframe()
# %%
views_df.head()

,created_at,description,display_name,is_public,name,report_id,report_path,showcased_at,spec,stars_count,user_id,view_count
0,2020-04-23 23:18:06+00:00,"In this article, we take a look at how to log and explore bounding boxes with Weights & Biases",Exploring Bounding Boxes for Object Detection With Weights & Biases,True,71dwgkt1rqy,87881,/stacey/yolo-drive/reports/--Vmlldzo4Nzg4MQ==,2020-11-02 17:17:56+00:00,"{""version"":5,""panelSettings"":{""smoothingWeight"":0,""xAxis"":""_step"",""ignoreOutliers"":false,""ref"":{""type"":""panelSettings"",""viewID"":""2qbvwmxcv"",""id"":""bm9v84lwr""}},""blocks"":[{""type"":""paragraph"",""children"":[{""text"":""""}]},{""type"":""paragraph"",""children"":[{""text"":""If you're training models for object detection, you can interactively visualize bounding boxes in Weights & Biases. This short demo focuses on driving scenes, testing a ""},{""type"":""link"",""children"":[{""text"":""YoloV3 net pretrained on MSCOCO""...",29,6329,14681
1,2020-12-11 13:44:42+00:00,How to version and interactively explore data and predictions across train/val/test with W&B's new Tables feature,Tables Tutorial: Visualize Data for Image Classification,True,idweeu53nfe,361760,/stacey/mendeleev/reports/--VmlldzozNjE3NjA=,2020-12-19 01:15:43+00:00,"{""version"":5,""panelSettings"":{""smoothingWeight"":0,""xAxis"":""_step"",""ignoreOutliers"":false,""ref"":{""type"":""panelSettings"",""viewID"":""ai9paen26"",""id"":""icqut5dkh""}},""blocks"":[{""type"":""paragraph"",""children"":[{""text"":""""}]},{""type"":""heading"",""children"":[{""text"":""Introduction ""}],""level"":1},{""type"":""paragraph"",""children"":[{""text"":""This is a walkthrough of dataset and prediction visualization using ""},{""type"":""link"",""children"":[{""text"":""Tables""}],""url"":""https://docs.wandb.ai/guides/tables"",""title"":null...",24,6329,14227
2,2021-07-18 23:34:12+00:00,"Generate images from a text prompt in this interactive report: DALL-E Mini Explained, a reproduction of OpenAI DALL·E",DALL-E Mini Explained,True,1e0cld9z8j4,862180,/dalle-mini/dalle-mini/reports/--Vmlldzo4NjIxODA=,2021-12-03 03:13:45+00:00,"{""version"":5,""panelSettings"":{""xAxis"":""train/step"",""smoothingWeight"":0,""smoothingType"":""exponential"",""ignoreOutliers"":false,""xAxisActive"":true,""smoothingActive"":false,""ref"":{""type"":""panelSettings"",""viewID"":""081wuiy9n"",""id"":""cpvu6o0i2""}},""blocks"":[{""type"":""paragraph"",""children"":[{""text"":""""}]},{""type"":""callout-block"",""children"":[{""type"":""callout-line"",""children"":[{""text"":""Watch ""},{""type"":""link"",""url"":""https://www.youtube.com/watch?v=vxc8FKqQxGM"",""children"":[{""text"":""The Story Behind DALL·E mi...",71,4119,143980
3,2020-10-08 01:12:14+00:00,"This article explains what you can do with our new Visualization Integrated Development Environment (IDE), and gives interactive examples so you can learn by doing.",The W&B Machine Learning Visualization IDE,True,nrnu0zw1mke,269778,/wandb/posts/reports/--VmlldzoyNjk3Nzg=,2020-10-11 21:28:24+00:00,"{""version"":5,""panelSettings"":{""smoothingWeight"":0,""xAxis"":""_step"",""ignoreOutliers"":false,""ref"":{""type"":""panelSettings"",""viewID"":""rrml4di6i"",""id"":""msz1ze703""}},""blocks"":[{""type"":""paragraph"",""children"":[{""emphasis"":true,""text"":""Read on to learn what you can do with our new Visualization IDE, and see interactive examples. Learn by doing? Try our ""},{""type"":""link"",""children"":[{""emphasis"":true,""text"":""example colab""}],""url"":""https://colab.research.google.com/drive/1uXLKDmsYg7QMRVFyjUAlg-eZH2MW8yW...",42,19,13331
4,2020-06-13 16:19:44+00:00,"In this article, we harness the latent power of autoencoders, one disentanglement at a time, and provide an example that you can try for yourself.",An Introduction to Adversarial Latent Autoencoders,True,iw1ivo7ld3h,140606,/authors/alae/reports/--VmlldzoxNDA2MDY=,2020-06-28 12:23:19+00:00,"{""version"":5,""panelSettings"":{""smoothingWeigh

In [28]:
# %%
def join_reports_data_to_lables(reports, tag_labels_matrix):
    reports['reportID'] = reports['report_path'].str.split('reports/--', 1, expand=True)[1]
    df = reports.merge(
        tag_labels_matrix, 
        how='left', 
        left_on='reportID', 
        right_index=True
    )
    return df

def extract_reports_text_as_column(reports_df, text_length_minimum=300):
    
    def findkeys(node, kv):
        if isinstance(node, list):
            for i in node:
                for x in findkeys(i, kv):
                   yield x
        elif isinstance(node, dict):
            if kv in node:
                yield node[kv]
            for j in node.values():
                for x in findkeys(j, kv):
                    yield x
                    
    def extract_text_from_report(df:pd.DataFrame(), index_in_df:int):
        spec = json.loads(df.iloc[index_in_df]['spec'])
        output = {
            'reportID': df.iloc[index_in_df]['reportID'],
        }

        text_found = ''

        if 'panelGroups' in spec:
            text_found = '\n'.join(
                [
                    x.get('content', None) 
                    for x in spec['panelGroups'] 
                    if (x.get('type', None) == 'markdown-block')
                ]
            )

        elif 'blocks' in spec:
            text_found = '\n'.join(
                [
                    str(x) for x in findkeys(spec, 'text')
                ]
            )

        output.update(text = text_found)    

        return output
    
    reports_text = [
        extract_text_from_report(reports_df, i) 
        for i in tqdm(range(len(reports_df)), desc="Text Extraction Progress")
    ]
    
    reports_text_df = pd.json_normalize(
        reports_text
    )
    
    df = (reports_text_df
        .assign(text = lambda x: x.text.str.replace("\n",""))
        .loc[reports_text_df.text.str.len() > text_length_minimum].merge(
            reports_df,
            on='reportID'
        )
        .drop('spec', axis=1)
    )
    
    return df

In [29]:
reports_data_df = (views_df
  .pipe(start_pipeline)
  .pipe(join_reports_data_to_lables, tag_labels_matrix=labels_matrix)
  .pipe(extract_reports_text_as_column, text_length_minimum=100)
)

/var/folders/x6/tdg2_lyn14n9vxhrw5znyrtw0000gn/T/ipykernel_38975/412858424.py:3: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  reports['reportID'] = reports['report_path'].str.split('reports/--', 1, expand=True)[1]
Text Extraction Progress: 100%|██████████| 521/521 [00:02<00:00, 201.01it/s]


In [40]:

# reports_data_df.dtypes


In [38]:

reports_data_df.head(2)

,reportID,text,created_at,description,display_name,is_public,name,report_id,report_path,showcased_at,stars_count,user_id,view_count,Articles,Course,Data,Domain,GPT2,Gradient Dissent,Has Colab,Level,Model,Normalization,OpenAI,Other,Task,Technique(s) or Industry (etc),Type,W&B Features
0,Vmlldzo4Nzg4MQ==,"If you're training models for object detection, you can interactively visualize bounding boxes in Weights & Biases. This short demo focuses on driving scenes, testing a YoloV3 net pretrained on MSCOCO on images from the Berkeley Deep Drive 100K dataset. The API for logging bounding boxes is flexible and intuitive. Below, I explain the interaction controls for this tool and a few ways you might use it to analyze your models. This approach can help in object detection on many other kinds of im...",2020-04-23 23:18:06+00:00,"In this article, we take a look at how to log and explore bounding boxes with Weights & Biases",Exploring Bounding Boxes for Object Detection With Weights & Biases,True,71dwgkt1rqy,87881,/stacey/yolo-drive/reports/--Vmlldzo4Nzg4MQ==,2020-11-02 17:17:56+00:00,29,6329,14681,NaN,NaN,[Berkeley Deep Drive],[Computer Vision],NaN,NaN,NaN,[Beginner],NaN,NaN,[YOLO],NaN,[Object Detection],[Autonomous Vehicles],[W&B Meta],[Panels]
1,VmlldzozNjE3NjA=,"Introduction This is a walkthrough of dataset and prediction visualization using Tables and Artifacts for image classification on W&B. Specifically, we'll finetune a convnet in Keras on photos from iNaturalist 2017 to identify 10 classes of living things (plants, insects, birds, etc). This is a tiny glimpse of how Tables can facilitate deep exploration and understanding of your data, models, predictions, and experiment progress, and you can find a variety of other examples here. Interact wi...",2020-12-11 13:44:42+00:00,How to version and interactively explore data and predictions across train/val/test with W&B's new Tables feature,Tables Tutorial: Visualize Data for Image Classification,True,idweeu53nfe,361760,/stacey/mendeleev/reports/--VmlldzozNjE3NjA=,2020-12-19 01:15:43+00:00,24,6329,14227,NaN,NaN,[iNaturalist],[Computer Vision],NaN,NaN,[Yes-Colab],[Intermediate],[Convnet],NaN,[Keras],[Chum here],[Classification],NaN,"[Experiment, W&B Meta]",[Tables]


In [ ]:
pprint(views_df.spec[50])

In [ ]:
def extract_keys_types(obj, prefix = ""):
    if isinstance(obj, dict):
        return {prefix + (f".{k}" if prefix else k): type(v).__name__ for k, v in obj.items()}
    elif isinstance(obj, list):
        return {prefix + f"[{i}]": type(v).__name__ for i, v in enumerate(obj)}
    else:
        return {prefix: type(obj).__name__}

def extract_all_keys_types(json_obj):
    keys_types = {}
    for k, v in json_obj.items():
        if isinstance(v, dict):
            keys_types.update(extract_all_keys_types(v))
        elif isinstance(v, list):
            for i, item in enumerate(v):
                if isinstance(item, (dict, list)):
                    keys_types.update(extract_all_keys_types({f"{k}[{i}]": item}))
                else:
                    keys_types[f"{k}[{i}]"] = type(item).__name__
        else:
            keys_types[k] = type(v).__name__
    return keys_types

spec = json.loads(views_df.spec[0])
keys_types = extract_all_keys_types(spec)
keys_types

In [122]:
import json

def spec_to_markdown(spec):
    spec_dict = json.loads(spec)
    markdown_text = ""
    for block in spec_dict["blocks"]:
        if block["type"] == "paragraph":
            for child in block["children"]:
                if "url" in child:
                    markdown_text += f"[{child['children'][0]['text']}]({child['url']})"
                elif "inlineCode" in child and child["inlineCode"]:
                    markdown_text += f"`{child.get('text', '')}`"
                else:
                    markdown_text += child.get("text", "")
            markdown_text += "\n\n"
        elif block["type"] == "heading":
            markdown_text += "#" * block["level"] + " " + block["children"][0]["text"] + "\n\n"
        elif block["type"] == "link":
            markdown_text += f"[{block['children'][0]['text']}]({block['url']})\n\n"
        elif block["type"] == "strong":
            markdown_text += f"**{block.get('text', '')}**\n\n"
        elif block["type"] == "list":
            for item in block["children"]:
                if item["type"] == "list-item":
                    for child in item["children"]:
                        if child["type"] == "paragraph":
                            for text_block in child["children"]:
                                if "inlineCode" in text_block and text_block["inlineCode"]:
                                    markdown_text += f"* `{text_block.get('text', '')}`"
                                else:
                                    markdown_text += f"* {text_block.get('text', '')}"
                            markdown_text += "\n"
            markdown_text += "\n"
        elif block["type"] == "code-block":
            markdown_text += "```\n"
            for line in block["children"]:
                markdown_text += line["children"][0].get("text", "") + "\n"
            markdown_text += "```\n\n"
        else:
            markdown_text += block.get("text", "") + "\n\n"
    return markdown_text

In [ ]:
# pprint()
pprint(spec_to_markdown(views_df.spec[500]))

# TODO
- REMOVE SHORT REPORTS

In [121]:
def convert_block_to_markdown(block):
    """
    Converts a single content block to its Markdown representation.
    """
    md_content = ""

    # Handle different types of blocks
    if block['type'] == 'paragraph':
        for child in block['children']:
            if 'url' in child:
                md_content += f"[{child['children'][0]['text']}]({child['url']})"
            elif 'inlineCode' in child and child['inlineCode']:
                md_content += f"`{child.get('text', '')}`"
            else:
                md_content += child.get('text', '')
        md_content += "\n\n"

    elif block['type'] == 'heading':
        md_content += "#" * block['level'] + " " + ''.join(child['text'] for child in block['children']) + "\n\n"

    elif block['type'] == 'list':
        for item in block['children']:
            if item['type'] == 'list-item':
                md_content += "* "
                for child in item['children']:
                    if child['type'] == 'paragraph':
                        for text_block in child['children']:
                            if 'inlineCode' in text_block and text_block['inlineCode']:
                                md_content += f"`{text_block.get('text', '')}`"
                            else:
                                md_content += text_block.get('text', '')
                md_content += "\n"
        md_content += "\n"

    elif block['type'] == 'code-block':
        md_content += "```\n"
        for line in block['children']:
            md_content += line['children'][0].get('text', '') + "\n"
        md_content += "```\n\n"

    elif block['type'] == 'block-quote' or block['type'] == 'callout-block':
        md_content += "\n> "
        for child in block['children']:
            md_content += child.get('text', '') + " "
        md_content += "\n\n"

    elif block['type'] == 'horizontal-rule':
        md_content += "\n---\n"

    elif block['type'] == 'latex':
        latex_content = ''.join(child.get('text', '') for child in block['children'])
        md_content += f"$$\n{latex_content}\n$$\n\n"

    # Other specific block types are ignored as per instructions

    return md_content

In [123]:
def spec_to_markdown(spec):
    spec_dict = json.loads(spec)
    markdown_text = ""
    for block in spec_dict["blocks"]:
        markdown_text += convert_block_to_markdown(block)
    return markdown_text

In [149]:
len(views_df[views_df["report_id"] == 334171]["spec"].values[0])

17446

In [152]:
views_df = pd.read_csv('report_data.csv')
views_df[views_df["report_id"] == 456265]

,Unnamed: 0,created_at,description,display_name,is_public,created_using,report_id,project_id,type,name,report_path,showcased_at,spec,stars_count,user_id,view_count
4,4,2021-02-08 15:31:00+00:00,NaN,Model 1+2,True,WANDB_UI,456265,213233,runs,d559kfkofv,/choke/wikilimo/reports/--Vmlldzo0NTYyNjU=,2021-02-09 09:20:51+00:00,"{""version"":4,""global"":{""name"":"""",""openViz"":true,""runSets"":[],""panelBankConfig"":{""state"":0,""settings"":{""autoOrganizePanels"":true,""showEmptySections"":false},""ref"":{""type"":""panel-bank-config"",""viewID"":""khpmvmdzl"",""id"":""cvulwvynd""},""sections"":[{""name"":""Hidden Panels"",""isOpen"":false,""type"":""flow"",""flowConfig"":{""snapToColumns"":true,""columnsPerPage"":3,""rowsPerPage"":2,""gutterWidth"":16,""boxWidth"":460,""boxHeight"":300},""ref"":{""type"":""panel-bank-section-config"",""viewID"":""khpmvmdzl"",""id"":""q5fphq4bv""},""pa...",0,71878,106


In [162]:
def extract_content_sequentially(json_obj):
    # Initialize an empty list to store the content
    content_list = []

    # Extract content under "panelGroups"
    for block in json_obj.get('panelGroups', []):
        content_list.append(block.get('content', ''))

    # Extract content under "global"
    global_block = json_obj.get('global', {})
    content_list.append(global_block.get('name', ''))

    return content_list

In [171]:
rep = views_df[views_df["report_id"] == 158496]
print(rep["display_name"], rep["report_path"])
ss = json.loads(rep["spec"].values[0])
pprint(ss)

10    Face Detection In VNIR Images: Results
Name: display_name, dtype: object 10    /adcoh/Vanilla Face Detectors - Evaluation/reports/--VmlldzoxNTg0OTY=
Name: report_path, dtype: object
{'authors': [{'name': 'Adam Cohn', 'username': 'adcoh'}],
 'discussionThreads': [],
 'global': {'customRunColors': {'ref': {'id': '3cw5y052u',
                                        'type': 'run-colors',
                                        'viewID': 'vr6fl0mf4'}},
            'name': '',
            'openViz': True,
            'panelBankConfig': {'ref': {'id': 'j2ntv5t67',
                                        'type': 'panel-bank-config',
                                        'viewID': 'vr6fl0mf4'},
                                'sections': [{'flowConfig': {'boxHeight': 300,
                                                             'boxWidth': 460,
                                                             'columnsPerPage': 3,
                                                          

In [170]:
pprint(extract_content_sequentially(ss))

['# Introduction\n'
 'This report is a summary of the Face Detection in NIR Images project, a '
 'Y-Data 2019 Industry Project.\n'
 '\n'
 'Below we will present the methodology we used and experiments we performed.\n'
 "We'll also present the results of the different pre-trained and re-trained "
 'Face Detection models on our target dataset.\n'
 '\n'
 '\n'
 '# Challenge\n'
 '\n'
 'Most Face Detection algorithms and use cases have been trained on RGB '
 'spectrum images, raising the question of their performance on non-RGB '
 'spectrum images for various tasks like Classification, Semantic '
 'Segmentation, Object Detection etc.\n'
 '\n'
 'Our mission was to develop an algorithm that could do Face Detection on '
 'Near-Infrared images, specifically on images of people in cars.\n'
 '\n'
 'The purpose of detecting faces in Near Infra-red images for our use case is '
 'to be able to count the number of people who are riding in a car in order to '
 'enforce the minimum person requirement fo

In [142]:
# pprint(spec_to_markdown(views_df.spec[0]))
pprint(spec_to_markdown(views_df[views_df["report_id"] == 334171]["spec"]))


TypeError: the JSON object must be str, bytes or bytearray, not Series

In [141]:
views_df[views_df["report_id"] == 121267]

,created_at,description,display_name,is_public,name,report_id,report_path,showcased_at,spec,stars_count,user_id,view_count
42,2020-05-26 16:33:22+00:00,,Adaption Eval 100 games vs 20 games,True,m9mj9chlvnb,121267,/bryonkucharski/TestProjects/reports/--VmlldzoxMjEyNjc=,2020-07-24 13:22:21+00:00,"{""version"":1,""global"":{""name"":"""",""openViz"":true,""runSets"":[],""panels"":{""views"":{""0"":{""name"":""Panels"",""defaults"":[],""config"":[],""panelRefs"":[]}},""tabs"":[""0""],""ref"":{""type"":""panels"",""viewID"":""uwb02tua3"",""id"":""aslergeie""}},""panelBankConfig"":{""state"":0,""settings"":{""autoOrganizePanels"":true,""showEmptySections"":false},""sections"":[{""name"":""Hidden Panels"",""isOpen"":false,""type"":""flow"",""flowConfig"":{""snapToColumns"":true,""columnsPerPage"":3,""rowsPerPage"":2,""gutterWidth"":16,""boxWidth"":460,""boxHeight"":300...",0,36422,91


In [118]:
ss = spec_dict = json.loads(views_df.spec[0])
pprint(convert_block_to_markdown(ss))

KeyError: 'type'

In [ ]:
reports_data_df = (views_df
  .pipe(start_pipeline)
  .pipe(join_reports_data_to_lables, tag_labels_matrix=labels_matrix)
  .pipe(extract_reports_text_as_column, text_length_minimum=100)
)

In [39]:
pprint(reports_data_df["description"][0])

('In this article, we take a look at how to log and explore bounding boxes '
 'with Weights & Biases')


In [ ]:
pprint(reports_data_df["text"][0])

In [34]:
# %%
reports_data_df.to_csv('reports_data_and_labels.csv')

In [ ]:
# %%
# !pip install wandb
# %%
import wandb
run = wandb.init(project='fc_recsys', entity='wandb-data-science')
# %%
artifact = wandb.Artifact('reports_data_and_lables', type='dataset')
artifact.add_file("reports_data_and_labels")

run.log_artifact(artifact)

In [ ]:
# %%
# testing low data reports
# %%
reports_df_L = (views_df
  .pipe(start_pipeline)
  .pipe(join_reports_data_to_lables, tag_labels_matrix=labels_matrix)
  .pipe(extract_reports_text_as_column, text_length_minimum=100)
)
# %%
reports_df_L = reports_df_L.assign(stripped_text = lambda x: x.text.str.replace("\n",""))
# %%
reports_df_L.sort_values(by="stripped_text", key=lambda x: x.str.len()).assign(stripped_text_len = lambda x: x.stripped_text.str.len())
# %%